In [2]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [3]:
INPUT_SIZE = 3 # thickness, height, angle
LEARNING_RATE = 0.001
BATCH_SIZE = 32 # 
NUM_EPOCHS = 50

TEST_SIZE = 0.2
RANDOM_STATE = 42
device = "cpu"

In [4]:
df = pd.read_csv(r'../MLP/processed_bending_stiffness.csv')

# Remove duplicates
initial_count = len(df)
df = df.drop_duplicates()
removed_count = initial_count - len(df)
if removed_count > 0:
    print(f"Removed {removed_count} duplicate row(s) from the dataset.")
print(f"Shape of dataset after removing duplicates: {df.shape}")
X = df[['Thickness', 'Height', 'Angle (deg)']]
y = df['Bending_Stiffness']

X_train, X_test, y_train, y_test = train_test_split(
    X.values, 
    y.values, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)
DATASET_SIZE = len(df) # Number of samples

Removed 685 duplicate row(s) from the dataset.
Shape of dataset after removing duplicates: (743, 4)


In [5]:
# Make all parameters have mean 0 and std 1 so parameters with a large scale don't skew 
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))  # Add .reshape(-1, 1)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))  # Add .reshape(-1, 1)

# Create DataLoaders
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)  # Now has shape [N, 1]
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)

In [23]:
class ConditionalVAE(nn.Module):
    def __init__(self, input_dim=3, condition_dim=1, hidden_dim=16, latent_dim=3, device=device):
        super(ConditionalVAE, self).__init__()

        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + condition_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2)
            )
        
        # latent space - mean and variance 
        self.mean_layer = nn.Linear(hidden_dim, latent_dim)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim)

        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, input_dim)
            )
     
    def encode(self, x, condition):
        x_with_condition = torch.cat([x, condition], dim=1)
        x = self.encoder(x_with_condition)
        mean, logvar = self.mean_layer(x), self.logvar_layer(x)
        return mean, logvar

    def reparameterization(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std).to(device)      
        z = mean + std * epsilon
        return z

    def decode(self, z, condition):
        # Concatenate latent with condition
        z = torch.cat([z, condition], dim=1)
        return self.decoder(z)

    def forward(self, x, condition):
        mean, logvar = self.encode(x, condition)
        z = self.reparameterization(mean, logvar)
        x_hat = self.decode(z, condition)
        return x_hat, mean, logvar

def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss + KLD

In [35]:
forward_model = torch.load("C:\\Users\\mason\\Work\\CMEC_SandwichPanel\\Models\\MLP\\Parameters_To_Stiffness\\full_model.pth")
forward_model.eval()

def custom_loss_function(x, x_hat, mean, log_var, target_stiffness, forward_model):
    # 1. Standard VAE Losses
    repro_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    
    print(f"X_hat: {x_hat}")
    # 2. Proxy/Consistency Loss
    # We pass the generated designs (x_hat) into the forward model
    # Note: x_hat is already scaled, which is what the MLP expects
    predicted_stiffness = forward_model(x_hat)
    
    # Compare MLP's prediction to the target stiffness given to the VAE
    proxy_loss = nn.functional.mse_loss(predicted_stiffness, target_stiffness, reduction='sum')
    
    # 3. Combine with weights (Alpha)
    # You might need to scale proxy_loss so it doesn't overpower the others
    alpha = 1.0 
    return repro_loss + KLD + (alpha * proxy_loss)

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch.nn.modules.container.Sequential was not an allowed global by default. Please use `torch.serialization.add_safe_globals([torch.nn.modules.container.Sequential])` or the `torch.serialization.safe_globals([torch.nn.modules.container.Sequential])` context manager to allowlist this global if you trust this class/function.

Check the documentation

In [24]:
model = ConditionalVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [25]:
def train(model, optimizer, epochs, device):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        total_samples = 0
        for batch_idx, (x, condition) in enumerate(train_loader):
            x = x.to(device)
            condition = condition.to(device)
            current_batch_size = x.size(0)

            optimizer.zero_grad()

            #Forward pass
            x_hat, mean, log_var = model(x, condition)
            loss = loss_function(x, x_hat, mean, log_var)
            
            overall_loss += loss.item()
            total_samples += current_batch_size
            
            # Backward pass
            loss.backward()
            optimizer.step()

        avg_loss = overall_loss / total_samples
        print("\tEpoch", epoch + 1, "\tAverage Loss: ", (avg_loss))
    return overall_loss

train(model, optimizer, epochs=NUM_EPOCHS, device=device)

	Epoch 1 	Average Loss:  3.1302884419759116
	Epoch 2 	Average Loss:  2.958769197817202
	Epoch 3 	Average Loss:  2.7794972846805046
	Epoch 4 	Average Loss:  2.62457249060223
	Epoch 5 	Average Loss:  2.3704276646829214
	Epoch 6 	Average Loss:  2.179405173870048
	Epoch 7 	Average Loss:  1.996323286884963
	Epoch 8 	Average Loss:  1.8972512447472774
	Epoch 9 	Average Loss:  1.771370621241303
	Epoch 10 	Average Loss:  1.7360547704728766
	Epoch 11 	Average Loss:  1.6835201244161586
	Epoch 12 	Average Loss:  1.6754373556837088
	Epoch 13 	Average Loss:  1.6797361341791122
	Epoch 14 	Average Loss:  1.6218953887220184
	Epoch 15 	Average Loss:  1.579169026127568
	Epoch 16 	Average Loss:  1.6154609481092255
	Epoch 17 	Average Loss:  1.5685001463199706
	Epoch 18 	Average Loss:  1.5061685099746243
	Epoch 19 	Average Loss:  1.473558753427833
	Epoch 20 	Average Loss:  1.5272336439652876
	Epoch 21 	Average Loss:  1.4431223628496883
	Epoch 22 	Average Loss:  1.5003837078107327
	Epoch 23 	Average Loss:  1

817.6113262176514

In [26]:
def generate(model, stiffness, x, y, samples, device='cpu'):
    model.eval()
    with torch.no_grad():
        normalized_stiffness = scaler_y.transform([[stiffness]])
        stiffness_tensor = torch.FloatTensor(normalized_stiffness).to(device) # Normalize 

        condition = stiffness_tensor.repeat(samples, 1) # Repeat for each sample

        z = torch.randn(samples, model.mean_layer.out_features).to(device)

        generated = model.decode(z, condition)

        generated_np = generated.cpu().numpy()
        designs = scaler_X.inverse_transform(generated_np)
    return designs

In [33]:
test_stiffness = 5000.0
designs = generate(model, test_stiffness, scaler_X, scaler_y, samples=10, device=device)
averageThickness = np.mean(designs[:, 0])
averageHeight = np.mean(designs[:, 1])
averageAngle = np.mean(designs[:, 2])
print(f"Average Thickness: {averageThickness:.2f}, Average Height: {averageHeight:.2f}, Average Angle: {averageAngle:.2f}")
#print("Generated designs for stiffness", test_stiffness, ":\n", designs)

Average Thickness: 5.22, Average Height: 76.15, Average Angle: 58.33


In [34]:
target = 5000.0

# Calculate the absolute difference
df['diff'] = (df['Bending_Stiffness'] - target).abs()

# Get the 5 rows with the smallest difference
closest_rows = df.nsmallest(5, 'diff')

print(closest_rows)

      Thickness  Height  Angle (deg)  Bending_Stiffness       diff
582       5.351  71.215         64.1        4990.091697   9.908303
6         3.190  93.433         63.3        4970.296544  29.703456
1244      7.490  58.942         60.1        5045.474938  45.474938
1206      7.395  61.011         42.7        5056.055142  56.055142
1044      6.804  61.683         57.4        4915.316049  84.683951
